In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
from zipfile import ZipFile
import urllib
from datetime import datetime
import string
import pygsheets
import requests
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import Bio.Entrez as Entrez
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.viennarna as viennarna
import paulssonlab.cloning.thermodynamics as thermodynamics
import paulssonlab.cloning.primers as primers
import paulssonlab.cloning.ncbi as ncbi

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Primers to make FP parts

In [ ]:
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
lib_parts = reg[("LIB", "parts")]
part_types = reg[("LIB", "parts", "Part types")]

In [ ]:
def overhangs_for(x):
    return (x["Upstream overhang"], x["Downstream overhang"])

In [ ]:
gg_overhangs = overhangs_for(part_types["CDS_CD"])

In [ ]:
storage_flanks = (
    lib_parts["JUMP_storage_vector_prefix"]["Sequence"],
    lib_parts["JUMP_storage_vector_suffix"]["Sequence"],
)

In [ ]:
ua_rbs = "tctagatttaagaaggagatatacat"
cluzel_cterm = "atgtccagacctgcaggcatgcaagctctagaggcat"
# flanks = (ua_rbs + "atg", "taa" + cluzel_cterm)

## Source plasmids

In [ ]:
%%time
plasmids = {
    row["Names"]: plib_maps[id_]
    for id_, row in plib_plasmids.items()
    if "cluzel-fp" in row["Tags"]
}

In [ ]:
plasmids.keys()

## Extract FP inserts

In [ ]:
%%time
locations = {
    name: sequence.amplicon_location(
        seq, ua_rbs, sequence.reverse_complement(cluzel_cterm)
    )
    for name, seq in plasmids.items()
}

In [ ]:
inserts = {name: seq.slice(*locations[name]) for name, seq in plasmids.items()}

In [ ]:
s = plasmids["pEB1-SCFP3A"]

In [ ]:
sequence.amplicon_location(s, flanks[0], sequence.reverse_complement(flanks[1]))

In [ ]:
s.slice(261, 972)

In [ ]:
print(str(_.seq))

In [ ]:
# check for BsmBI/BsaI/BbsI/AarI

In [ ]:
plasmids["pEB1-SCFP3A"]

In [ ]:
for enzyme_name in ("BsaI", "BsmBI", "BbsI", "AarI"):
    names_with_cuts = []
    for name, seq in inserts.items():
        cuts = enzyme.re_search(seq, enzyme_name)
        if cuts:
            names_with_cuts.append(name)
    print(f"{enzyme_name} ({len(names_with_cuts)}): {', '.join(names_with_cuts)}")

## Find FP common ends

In [ ]:
names = np.array(list(inserts.keys()))

In [ ]:
max_end_length = 40

In [ ]:
letters = np.array([list(s.seq_lower()[:max_end_length]) for s in inserts.values()])
counts = (letters[np.newaxis, :, :] == letters[:, np.newaxis, :]).sum(axis=0)
forward_cumulative_counts = np.minimum.accumulate(counts, axis=1)

In [ ]:
reverse_letters = np.array(
    [list(s.seq_lower()[-max_end_length:]) for s in inserts.values()]
)[:, ::-1]
reverse_counts = (
    reverse_letters[np.newaxis, :, :] == reverse_letters[:, np.newaxis, :]
).sum(axis=0)
reverse_cumulative_counts = np.minimum.accumulate(reverse_counts, axis=1)

In [ ]:
cumulative_counts = np.minimum(forward_cumulative_counts, reverse_cumulative_counts)

In [ ]:
majority_size = cumulative_counts.max(axis=0)

In [ ]:
majority_size

In [ ]:
idxs = np.concatenate(
    (np.where(np.diff(majority_size) != 0)[0], [len(majority_size) - 1])
)

In [ ]:
idxs

In [ ]:
majority_size[idxs]

In [ ]:
for idx in idxs:
    omitted = names[cumulative_counts[:, idx] < majority_size[idx]]
    omitted_str = ", ".join(omitted)
    print(f"length {idx} all except ({len(omitted)}): {omitted_str}")

In [ ]:
idx = 13
names[cumulative_counts[:, idx] < majority_size[idx]]

## Design primers

In [ ]:
import primer3plus

In [ ]:
# USE CASES:
# 1) take desired product, template seq, find overhangs
# 2) take amplicon, optional overhangs
def primer3_amplicon_primers(
    template, flanks, tm=(55, 65, 72), return_explain=False, return_many=False
):
    template_seq = str(sequence.get_seq(template)).lower()
    full_flanks = workflow.concatenate_flanks(*flanks)
    trimmed_flanks = workflow.smoosh_and_trim_flanks(template_seq, full_flanks)
    design = primer3plus.Design()
    design.settings.template(template_seq)
    design.settings.as_cloning_task()
    design.settings.use_overhangs()
    design.settings.left_overhang(trimmed_flanks[0])
    design.settings.right_overhang(trimmed_flanks[1])
    if len(tm) != 3:
        raise ValueError("expecting (min, optimal, max) tm")
        design.params["PRIMER_OPT_TM"] = tm[0]
        design.params["PRIMER_MIN_TM"] = tm[1]
        design.params["PRIMER_MAX_TM"] = tm[2]
    design.settings.product_size([27, 10000], opt=0)
    if return_many is False:
        num_return = 1
    else:
        num_return = return_many
    design.settings.primer_num_return(num_return)
    # print(design.params)
    results, explain = design.run()
    if not return_many:
        if len(results) == 0:
            raise ValueError("did not design primers")
        elif len(results) > 1:
            raise NotImplementError
        # return primers.PrimerPair(primer3=results[0])
        return results
    else:
        return results
        # return [primers.PrimerPair(primer3=r) for r in results.values()]
    return results, explain


primer3_amplicon_primers(
    inserts["pEB1-SCFP3A"], [gg_overhangs, storage_flanks], return_many=3
)

In [ ]:
primer3pairtest = {
    "PAIR": {
        "PENALTY": 17.00987790954747,
        "COMPL_ANY_TH": 8.920937018501036,
        "COMPL_END_TH": 0.3357452745337355,
        "PRODUCT_SIZE": 717,
    },
    "LEFT": {
        "PENALTY": 8.214364424255905,
        "SEQUENCE": "atgagtaaaggagaagaacttttcact",
        "location": (0, 27),
        "TM": 58.785635575744095,
        "GC_PERCENT": 33.333333333333336,
        "SELF_ANY_TH": 7.255354561467641,
        "SELF_END_TH": 0.0,
        "HAIRPIN_TH": 37.94059695097741,
        "END_STABILITY": 3.41,
        "OVERHANG": "gcttcacgtctcggtctcaa",
    },
    "RIGHT": {
        "PENALTY": 8.795513485291565,
        "SEQUENCE": "ttatttgtatagttcatccatgccatg",
        "location": (716, 27),
        "TM": 58.204486514708435,
        "GC_PERCENT": 33.333333333333336,
        "SELF_ANY_TH": 0.0,
        "SELF_END_TH": 0.0,
        "HAIRPIN_TH": 0.0,
        "END_STABILITY": 3.66,
        "OVERHANG": "ggttgagaccggagacgtgctaa",
    },
}
primer3test = primer3pairtest["LEFT"]

In [ ]:
p1 = primers.Primer(primer3=primer3test)

In [ ]:
a = primers.PrimerPair("aaaagggg", "ccccaaaa")
b = primers.PrimerPair("aaaagggg", "ccccaaaa", template="gggg")
c = primers.PrimerPair(
    Primer("gggg" + ("a" * 10), binding_length=10),
    Primer("gggg" + ("a" * 10), binding_length=10),
)
d = primers.PrimerPair(primer3=primer3pairtest)

In [ ]:
a = Primer("gggg" + ("a" * 10), binding_length=10)
b = Primer("gggg" + ("a" * 10), overhang_length=4)
c = Primer("gggg", "a" * 10)
d = Primer("gggg" + ("a" * 10))
e = Primer("gggg" + ("a" * 10), template="ccccaaaacccc")
f = Primer(binding="a" * 10, overhang="gggg")
g = Primer("gggg", ("a" * 10), template="ccccaaaacccc")  # error
h = Primer(primer3=primer3testresult)
h2 = Primer(template="ggggaaaagggg", primer3=primer3testresult)  # error
print(a, b, c, d, e, f, h, h2)

In [ ]:
p = primers.Primer("gggg" + ("a" * 10), template="cccc" + ("a" * 10) + "tttt")

In [ ]:
p

In [ ]:
p.tm

In [ ]:
p.mfe_monomer

In [ ]:
primers.Primer("gggg" + ("a" * 10), binding_length=10)